# Starting our Docker Environment
### We even use Gradle to manage Docker using `com.avast.gradle.docker-compose` plugin:

In [ ]:
!./gradlew --console=plain tasks --group docker -q
!./gradlew --console=plain -q composeUp

### Using Python to interact with KSQL. Let's see if `CLICKSTREAM`, `CLICKSTREAM_CODES` and `CLIICKSTREAM_USERS` are there:

In [ ]:
ksql = "/usr/local/bin/ksql"
!echo "LIST TOPICS;" | "$ksql"

# The Developer Experience
### Create a *registration* stream `CLICKSTREAM`. This is sort of like DDL in relational DBs:

In [ ]:
sql = (
    "CREATE STREAM clickstream "
    "(_time bigint, time varchar, ip varchar, request varchar, status int, userid int, bytes bigint, agent varchar) "
    "with (kafka_topic = 'clickstream', value_format = 'json');"
)
!echo "$sql" | "$ksql"

### Create *persistent query* table `EVENTS_PER_MIN`, which reads data from `CLICKSTREAM` and initializes the streaming process:

In [ ]:
sql = (
    "CREATE TABLE events_per_min AS SELECT userid, count(*) AS events "
    "FROM clickstream window "
    "TUMBLING (size 60 second) GROUP BY userid;"
)
!echo "$sql" | "$ksql"

### Suppose we want to make changes to `CLICKSTREAM`. Just drop it right?

In [ ]:
sql = (
    "DROP STREAM clickstream; "
)
!echo "$sql" | "$ksql"

### Bummer. First we need to terminate the persistent query... but the *query id* is auto-incrementing, and not constant:

In [ ]:
sql = (
    "TERMINATE QUERY CTAS_EVENTS_PER_MIN_0; "
)
!echo "$sql" | "$ksql"

### Now drop CLICKSTREAM again

In [ ]:
sql = (
    "DROP STREAM clickstream; "
)
!echo "$sql" | "$ksql"

### We can do something better to increase developer efficiency!

# The `gradle-confluent` Plugin

### [GitHub Repository](https://github.com/RedPillAnalytics/gradle-confluent)

### [Gradle Plugin Portal](https://plugins.gradle.org/plugin/com.redpillanalytics.gradle-confluent)

### Our `build.gradle` file:

### Let's take a look at the Gradle tasks available by applying the plugin:

In [ ]:
!./gradlew ksql:tasks --group confluent

### Let's see the *help* assocated with the `pipelineExecute` task:

In [ ]:
!./gradlew help --task ksql:pipelineExecute

### Let's take a look at our [pipeline source code](https://github.com/RedPillAnalytics/kafka-examples/tree/master/ksql/src/main/pipeline).

### Used to execute a single pipeline. Notice the plugin auto-generates `TERMINATE` and `DROP` statements:

In [ ]:
!./gradlew --console=plain ksql:pipelineExecute --pipeline-dir 01-clickstream

### Or multiple pipelines. We'll turn the logging up a bit. The `DROP` statements occur in reverse order:

In [ ]:
!./gradlew --console=plain ksql:pipelineExecute -i

### The `pipelineExecute` task is great for developers. But in a real delivery pipeline, we want to build and publish artifacts. We are using mavenLocal() which defaults to `$HOME/.m2` as our maven repository location:

In [ ]:
!./gradlew --console=plain ksql:build ksql:publish

In [ ]:
!zipinfo ~/.m2/repository/com/redpillanalytics/ksql-pipeline/1.0.0/ksql-pipeline-1.0.0.zip

### We build the `ksql-script.sql` script in case we want to use it as our queries file, with either option `--queries-file` or, with property `ksql.queries.file`. Notice we *normalize* the script, removing comments, line breaks, etc.:

In [ ]:
!cat ksql/build/pipeline/ksql-script.sql

### We chose to deploy our artifacts using the KSQL REST API, which is similar to using the `pipelineExecute` command, but extracts and executes scripts in the artifact instead:

In [ ]:
!./gradlew --console=plain ksql:deploy

### Again, with `-i` for more info:

In [ ]:
!./gradlew --console=plain ksql:deploy -i

### We realized quickly that we needed to be able to granularly control some of our auto-generated KSQL. For instance... we wanted to control whether `DELETE TOPIC` was added to `DROP` statements. So we introduced *directives*:

### Directives are *smart comments* beginning with `--@`. So far we have only introduced `--@DeleteTopic`, but others are planned:

In [ ]:
!./gradlew --console=plain ksql:pipelineExecute --pipeline-dir 01-clickstream -i

# KSQL User Defined Functions

### What we really needed was a `CASE` statement. KSQL doesn't have the [CASE](https://github.com/confluentinc/ksql/issues/620) yet, but it's coming.

### In the meantime, we wrote the `decode()` function, which was inspired by the Oracle `decode()` function that existed before `CASE`.

### `Decode` function:

```groovy
import groovy.util.logging.Slf4j
import io.confluent.ksql.function.udf.Udf
import io.confluent.ksql.function.udf.UdfDescription

@Slf4j
@UdfDescription(
        name = "decode",
        description = """Given up to 3 pairs of 'search' and 'text', return the first 'text' value where 'search' matches 'expression'. If no match, return 'defaultvalue'. 'ignorecase' defaults to 'false'.""")
class Decode {

   @Udf(description = """Given 1 pair of 'search' and 'text', return the first 'text' value where 'search' matches 'expression'. If no match, return 'defaultvalue'. 'ignorecase' defaults to 'false'.""")
   String decode(String expression, String search1, String text1, String defaultvalue, Boolean ignorecase = false) {

      // If any of the expected values are null, then just return null
      if (expression == null || search1 == null || text1 == null) return null

      return Utils.textMatch(expression, search1, ignorecase) ? text1 : defaultvalue
   }
    
   @Udf(description = """Given 2 pairs of 'search' and 'text', return the first 'text' value where 'search' matches 'expression'. If no match, return 'defaultvalue'. 'ignorecase' defaults to 'false'.""")
   String decode(String expression, String search1, String text1, String search2, String text2, String defaultvalue, Boolean ignorecase = false) {

      // If any of the expected values are null, then just return null
      if (expression == null || search1 == null || text1 == null || search2 == null || text2 == null) return null

      if (Utils.textMatch(expression, search1, ignorecase)) return text1

      else if (Utils.textMatch(expression, search2, ignorecase)) return text2

      else return defaultvalue
   }
    
   @Udf(description = """Given 3 pairs of 'search' and 'text', return the first 'text' value where 'search' matches 'expression'. If no match, return 'defaultvalue'. 'ignorecase' defaults to 'false'.""")
   String decode(String expression, String search1, String text1, String search2, String text2, String search3, String text3, String defaultvalue, Boolean ignorecase = false) {

      // If any of the expected values are null, then just return null
      if (expression == null || search1 == null || text1 == null || search2 == null || text2 == null || search3 == null || text3 == null) return null

      if (Utils.textMatch(expression, search1, ignorecase)) return text1

      else if (Utils.textMatch(expression, search2, ignorecase)) return text2

      else if (Utils.textMatch(expression, search3, ignorecase)) return text3

      else return defaultvalue
   }
}
```

### `DecodeTest` Spock test specification:

```groovy
import groovy.util.logging.Slf4j
import spock.lang.Shared
import spock.lang.Specification
import spock.lang.Unroll

@Slf4j
class DecodeTest extends Specification {

   @Shared
   Decode decode = new Decode()

   @Unroll
   def "When: #expression, #search, #text, #defaultValue; Expect: #result"() {

      expect:
      decode.decode(expression, search, text, defaultValue) == result

      where:
      expression    | search        | text  | defaultValue || result
      'KSQL Rocks!' | 'ksql rocks!' | 'yes' | 'no'         || 'no'
      'KSQL Rocks!' | 'KSQL Rocks!' | 'yes' | 'no'         || 'yes'
   }

   @Unroll
   def "When: #expression, #search, #text, #defaultValue, #ignorecase; Expect: #result"() {

      expect:
      decode.decode(expression, search, text, defaultValue, ignorecase) == result

      where:
      expression    | search        | text  | defaultValue | ignorecase || result
      'KSQL Rocks!' | 'KSQL rocks!' | 'yes' | 'no'         | true       || 'yes'
      'KSQL Rocks!' | 'KSQL Rocks!' | 'yes' | 'no'         | true       || 'yes'
      'KSQL Rocks!' | 'KSQL rocks!' | 'yes' | 'no'         | false      || 'no'
   }

   @Unroll
   def "When: #expression, #search1, #text1, #search2, #text2, #defaultValue, #ignorecase; Expect: #result"() {

      expect:
      decode.decode(expression, search1, text1, search2, text2, defaultValue, ignorecase) == result

      where:
      expression    | search1       | text1 | search2       | text2 | defaultValue | ignorecase || result
      'KSQL Rocks!' | 'KSQL rocks!' | 'yes' | 'KSQL Sucks!' | 'no'  | 'no'         | true       || 'yes'
      'KSQL Rocks!' | 'KSQL rocks!' | 'no'  | 'KSQL Rocks!' | 'yes' | 'no'         | false      || 'yes'
      'KSQL Rocks!' | 'KSQL rocks!' | 'no'  | 'KSQL Sucks!' | 'no'  | 'yes'        | false      || 'yes'
   }

   @Unroll
   def "When: #expression, #search1, #text1, #search2, #text2, #search3, #text3, #defaultValue, #ignorecase; Expect: #result"() {

      expect:
      decode.decode(expression, search1, text1, search2, text2, search3, text3, defaultValue, ignorecase) == result

      where:
      expression    | search1       | text1 | search2       | text2 | search3       | text3 | defaultValue | ignorecase || result
      'KSQL Rocks!' | 'KSQL rocks!' | 'yes' | 'KSQL Sucks!' | 'no'  | 'KSQL, meh'   | 'no'  | 'no'         | true       || 'yes'
      'KSQL Rocks!' | 'KSQL rocks!' | 'no'  | 'KSQL Rocks!' | 'yes' | 'KSQL, meh'   | 'no'  | 'no'         | false      || 'yes'
      'KSQL, meh'   | 'KSQL rocks!' | 'no'  | 'KSQL Sucks!' | 'no'  | 'KSQL, meh'   | 'yes' | 'no'         | false      || 'yes'
      'KSQL, meh'   | 'KSQL rocks!' | 'no'  | 'KSQL Sucks!' | 'no'  | "What's KSQL" | 'no'  | 'yes'        | false      || 'yes'
      'KSQL, meh'   | 'ksql rocks!' | 'no'  | 'ksql, meh'   | 'yes' | "What's KSQL" | 'no'  | 'no'         | true       || 'yes'

   }
}
```

### And here is our `build.gradle` file for the `functions` subproject:

### Let's build and publish our UDF artifact, and see how many items are in it:

In [ ]:
!./gradlew --console=plain functions:build functions:publish
!zipinfo -h ~/.m2/repository/com/redpillanalytics/functions/1.0.0/functions-1.0.0.jar

In [ ]:
!/usr/local/bin/docker cp ~/.m2/repository/com/redpillanalytics/functions/1.0.0/functions-1.0.0.jar ksql-server:/etc/ksql-server/ext

In [ ]:
!echo "DESCRIBE FUNCTION DECODE;" | "$ksql"

In [ ]:
sql = (
    "select definition, decode(definition, "
    "'Proxy authentication required','Bad', "
    "'Page not found','Bad', "
    "'Redirect','Good', "
    "'Unknown') label "
    "from enriched_error_codes limit 20;"
)
!echo "$sql" | "$ksql"

# Stopping our Docker environment

In [ ]:
!./gradlew --console=plain -q composeDown
!/usr/local/bin/docker ps